In [21]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import csv
import re

filenames = glob.glob('C:/Users/hatic/Downloads/ORCID/ORCID/*/*/educations/*.xml')
# dosyalarınızın olduğu en ana klasörün yolunu "/*/*/educations/*.xml" kısmından öncesine yapıştırın.
# ilk "*" "000" formatında başlayan 3 haneli klasörlerin tamamını okumak için
# ikinci "*" orchid id başlıklı klasörlerin tamamını okuması için
# üçüncü "*" kategori başlıklı klasörlerin tamamını okuması için
# dördüncü "*.xml" klasörlerin altındaki tüm XML dosyalarını okuması için 

orchid_xml_data = pd.DataFrame()

for filename in filenames:
    with open(filename, "r", encoding="utf-8") as content:
        trees = ET.parse(content)
        root = trees.getroot()
        tags = [elem.tag for elem in root.iter()]
        columnnames = [re.sub(r'{.*}', '', s) for s in tags]
        texts = [elem.text for elem in root.iter()]
        rows = []
        xml_dict = dict(zip(columnnames, texts))
        rows.append(xml_dict)
        df = pd.DataFrame(rows, columns=columnnames)
        df["filename"] = filename
        df = df.loc[:,~df.columns.duplicated()].reset_index(drop=True)
        orchid_xml_data = pd.concat([orchid_xml_data , df] , axis = 0 , ignore_index = True)
    
drop_cols = ["education", 'source', 'source-orcid', 'start-date', 'end-date',
'organization', 'address', 'disambiguated-organization', "source-client-id"]
orchid_xml_data.drop(columns=drop_cols, inplace=True)
orchid_xml_data.rename(columns={"path": "orcid-id", "name": "education", "country": "country-code", 'credit-name': "contributor"}, inplace=True)
orchid_xml_data["source-name"] = orchid_xml_data["source-name"].fillna("").astype("object")
orchid_xml_data["month"] = orchid_xml_data["month"].fillna(0).astype("int64")
orchid_xml_data["year"] = orchid_xml_data["year"].fillna(0).astype("int64")
orchid_xml_data["day"] = orchid_xml_data["day"].fillna(0).astype("int64")
orchid_xml_data['created-date'] = orchid_xml_data['created-date'].astype("datetime64")
orchid_xml_data['last-modified-date'] = orchid_xml_data['last-modified-date'].astype("datetime64")
temp_cols=orchid_xml_data.columns.tolist()
new_cols=temp_cols[-1:] + temp_cols[:-1]
orchid_xml_data=orchid_xml_data[new_cols]
temp_cols = orchid_xml_data.columns.tolist()
index = orchid_xml_data.columns.get_loc("filename")
new_cols=temp_cols[index:index+1] + temp_cols[0:index] + temp_cols[index+1:]
orchid_xml_data =orchid_xml_data[new_cols]
orchid_xml_data.index += 1
orchid_xml_data.to_csv('C:/Users/hatic/Downloads/ORCID/ORCID/educations.csv', index_label="xml-id", sep="^") # csvyi kaydetmek istediğiniz konumu ve ismi yazın
